In [28]:
%use dataframe, lets-plot

In [26]:
@DataSchema
interface ChampionBanStats {
    @ColumnName("CHAMPION")
    val champion: String?
    
    @ColumnName("BAN %")
    val banRate: String?
    
    @ColumnName("TOTAL")
    val total: Int?
    
    @ColumnName("BLUE")
    val blue: Int?
    
    @ColumnName("RED")
    val red: Int?
    
    @ColumnName("P1 TOTAL")
    val totalPick1: Int?
    
    @ColumnName("P1 BLUE")
    val bluePick1: Int?
    
    @ColumnName("P1 RED")
    val redPick1: Int?
    
    @ColumnName("P2 TOTAL")
    val totalPick2: Int?
    
    @ColumnName("P2 BLUE")
    val bluePick2: Int?
    
    @ColumnName("P2 RED")
    val redPick2: Int?
}

In [68]:
import org.jetbrains.letsPlot.intern.Plot

val banStatsDataFrame = DataFrame.readCSV("resources/LCK_summer_playoffs_2023_13.14_champion_ban_stats.csv").cast<ChampionBanStats>()

fun List<Pair<String?, Int?>>.toChartData() =
    sortedByDescending { it.second }
        .take(5)
        .toMap()
        .let {
            mapOf(
                "name" to it.keys,
                "value" to it.values
            )
        }

val blueChartData = banStatsDataFrame
    .map { champion to bluePick1 }
    .toChartData()

val redChartData = banStatsDataFrame
    .map { champion to redPick1 }
    .toChartData()

fun createPieChart(data: Map<String, Collection<Any?>>, title: String): Plot {
    val (w, h) = 600 to 250
    val blankTheme = theme(line = elementBlank(), axis = elementBlank())
    return letsPlot(data) +
            ggtitle(title) + 
            ggsize(w, h) +
            geomPie(
                size = 30,
                stroke = 1,
                strokeColor = "white",
                hole = 0,
                labels = layerLabels().line("@name").size(13),
                stat = Stat.identity,
            ) { slice = "value"; fill = "name" } +
            theme().legendPositionNone() +
            blankTheme +
            scaleFillBrewer(palette = "Set1")
}

val blueChart = createPieChart(blueChartData, "Blue 1픽 Top 5")
val redChart = createPieChart(redChartData, "Red 1픽 Top 5")

gggrid(
    plots = listOf(
        blueChart,
        redChart
    )
)
